# Download of libraries and data

In [9]:
import os
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib as mpl

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions


tfk = tf.keras
tfkl = tf.keras.layers

In [10]:
#Directory for the dataset
dataset_dir="../input/plants/training"

# Define the batch size
batch_size = 256

# Versioning
version_number="0.5"
version_name="transferLearning_inceptionv3"
model_name = version_name + "_" + version_number

In [11]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Creating the model

In [12]:
#Instead of using the ReLU as activation function we decided to apply the Leaky ReLU
from keras.layers import LeakyReLU
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01)

# Load Inception V3
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(256,256,3))

# Create the model: adding Dense layers and Dropout layers
model = tf.keras.Sequential()
model.add(inception)
model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.35))
model.add(tf.keras.layers.Dense(1024, activation=lrelu))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation=lrelu))
model.add(tf.keras.layers.Dropout(0.35))
model.add(tf.keras.layers.Dense(256, activation=lrelu)) 
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation=lrelu))
model.add(tf.keras.layers.Dropout(0.35))
model.add(tf.keras.layers.Dense(14, activation='softmax'))

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.5e-5), metrics='accuracy')
# model.summary()

In [13]:
# Setting "not trainable" the first 50 layers of the model. We tried few different values and 50 resulted as a good choice.

for layer in inception.layers:
        layer.trainable = True
for layer in inception.layers[:50]:
        layer.trainable = False

model.summary()

# Processing of dataset and creation of Generators

In [ ]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
                                        rotation_range=30,
                                        height_shift_range=25,
                                        width_shift_range=35,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        validation_split=0.1,
                                        preprocessing_function=preprocess_input
                                       )


# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                                       target_size=(256,256),
                                                       color_mode='rgb',
                                                       classes=None,
                                                       class_mode='categorical',
                                                       shuffle=True,
                                                       seed=seed,
                                                       subset='training')
valid_gen =  aug_train_data_gen.flow_from_directory(directory=dataset_dir, 
                                               target_size=(256, 256),
                                               color_mode='rgb',
                                               classes= None,
                                               shuffle=False,
                                               seed=seed,
                                               subset='validation')

# Training of the model

In [ ]:
# Train the model for 200 epochs
history = model.fit(
    x = aug_train_gen,
    batch_size = batch_size,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
# Save best epoch model
model.save("models/" + model_name)

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()